# Part 1: Training a Convolutional Neural Network

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from utils import * 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

In [2]:
cwd = os.getcwd()

train_path = os.path.join(cwd,'Data','train_val')
test_path  = os.path.join(cwd,'Data','test')

In [3]:
input_train = xr.open_dataset(os.path.join(train_path,'forcing_train.nc'))
input_test = xr.open_dataset(os.path.join(test_path,'forcing_test.nc'))


In [4]:
output_train = xr.open_dataset(os.path.join(train_path,'member_165_train.nc'))
output_test = xr.open_dataset(os.path.join(test_path,'member_165_test.nc'))

output_train = output_train.sel(x=slice(-3024000,3040000), y=slice(-3024000,3040000))
output_test = output_test.sel(x=slice(-3024000,3040000), y=slice(-3024000,3040000))

In [5]:
X_train_df = pd.DataFrame({"delta_T": input_train.delta_T.data},index=input_train.delta_T.coords['time'].data)

In [6]:
y_train = output_train.velsurf_mag.fillna(0).data

In [7]:
X_train = X_train_df.to_numpy()

In [8]:
print(X_train.shape,y_train.shape)

(1049, 1) (1049, 381, 381)


In [9]:
from tensorflow.keras import backend as K

def custom_rmse(y_true, y_pred):
    """
    custom_rmse(y_true, y_pred)
    calculates root square mean value with focusing only on the ocean
    """
    y_pred = y_pred[(y_true != 0) & (y_true != 0.0)]
    y_true = y_true[(y_true != 0) & (y_true != 0.0)]
    
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    return K.sqrt(K.mean(tf.math.squared_difference(y_pred, y_true),axis= -1))

In [10]:
n_filters  = 9  # number of filters
n_neurons  = 32  # number of neurons in the Dense layer
activation     = 'relu' # activation function
kernel_size    = 3
learning_rate  = 0.01
minibatch_size = 64
num_epochs     = 100

In [11]:
model = Sequential()

model.add(Dense(n_filters*127*127, input_shape=(X_train.shape[1],), activation=activation)) # shape: (6912,1)
model.add(Reshape((127,127,n_filters))) # shape: (12,18,32)

# Upsample to 96x144
model.add(Conv2DTranspose(filters=n_filters, kernel_size=kernel_size, 
                          activation=activation, strides=3, padding='same')) # shape: (96,144,32)

model.add(Conv2DTranspose(filters=1, kernel_size=kernel_size, activation="linear", padding="same")) # shape: (96,144,1)


model.compile(loss=custom_rmse,optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

2022-11-16 14:19:44.584360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 145161)            290322    
                                                                 
 reshape (Reshape)           (None, 127, 127, 9)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 381, 381, 9)      738       
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 381, 381, 1)      82        
 ranspose)                                                       
                                                                 
Total params: 291,142
Trainable params: 291,142
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)


history = model.fit(X_train, y_train, 
                    batch_size      = minibatch_size,
                    epochs          = num_epochs,
                    validation_split= 0.2, 
                    verbose         = 1,
                    callbacks       = [early_stop])



Epoch 1/100
14/14 [==============================] - 18s 1s/step - loss: 185.8925 - val_loss: 124.4899
Epoch 2/100
14/14 [==============================] - 17s 1s/step - loss: 183.7090 - val_loss: 116.0143
Epoch 3/100
14/14 [==============================] - 19s 1s/step - loss: 172.3245 - val_loss: 105.2388
Epoch 4/100
14/14 [==============================] - 30s 2s/step - loss: 153.4816 - val_loss: 120.5420
Epoch 5/100
14/14 [==============================] - 16s 1s/step - loss: 138.4327 - val_loss: 110.8876
Epoch 6/100
14/14 [==============================] - 18s 1s/step - loss: 127.8453 - val_loss: 111.6966
Epoch 7/100
14/14 [==============================] - 17s 1s/step - loss: 120.0449 - val_loss: 106.1059
Epoch 8/100
14/14 [==============================] - 22s 2s/step - loss: 113.7906 - val_loss: 101.7658
Epoch 9/100
14/14 [==============================] - 18s 1s/step - loss: 109.9615 - val_loss: 100.2386
Epoch 10/100
14/14 [==============================] - 21s 1s/step - loss:

In [ ]:
plot_history(history)

In [ ]:
model_path = os.path.join(cwd,'saved_model')
make_dir(model_path)

In [ ]:
model.save(os.path.join(model_path,'CNN_model_1_member.h5'))

## Testing

In [ ]:
model = load_model(os.path.join(model_path,'CNN_model_1_member.h5'),custom_objects={'custom_rmse':custom_rmse})

In [ ]:
X_test_df = pd.DataFrame({"delta_T": input_test.delta_T.data},index=input_test.delta_T.coords['time'].data)

In [ ]:
y_test_df = output_test.velsurf_mag.fillna(0)

In [ ]:
X_test = X_test_df.to_numpy()


In [ ]:
y_test_pre = model.predict(X_test)


In [ ]:
y_test_pre_rs = y_test_pre.reshape(y_test_df.shape[0],y_test_df.shape[1],y_test_df.shape[2])

y_test_pre_xr = xr.Dataset(coords={'time': X_test_df.index.values, 
                               'x': y_test_df.x.values, 
                               'y': y_test_df.y.values},
                       data_vars=dict(velsurf_mag=(['time', 'y', 'x'], y_test_pre_rs)))

In [ ]:
y_test_pre_xr

In [ ]:
y_true =  y_test_df

In [ ]:
y_true

In [ ]:
y_test_pre_xr.velsurf_mag.mean(dim=['x','y']).plot()
y_true.mean(dim=['x','y']).plot()

In [ ]:
(y_true-y_test_pre_xr.velsurf_mag).mean(dim='time').plot()

In [ ]:
y_true.mean(dim='time').plot()

In [ ]:
y_test_pre_xr.velsurf_mag.mean(dim='time').plot()

In [ ]:
fig, axes = plt.subplots(figsize=(15,12),ncols=2,nrows=3)

yrs = [-19800, -10000, 0]
vmin, vmax    = 0, 2500
cmap = 'RdBu_r'
y_test_pre_xr.velsurf_mag.sel(time=yrs[0]).plot(ax=axes[0,0], vmin=vmin, vmax=vmax,cmap=cmap)
y_true.sel(time=yrs[0]).plot(ax=axes[0,1], vmin=vmin, vmax=vmax,cmap=cmap)

y_test_pre_xr.velsurf_mag.sel(time=yrs[1]).plot(ax=axes[1,0], vmin=vmin, vmax=vmax,cmap=cmap)
y_true.sel(time=yrs[1]).plot(ax=axes[1,1], vmin=vmin, vmax=vmax,cmap=cmap)

y_test_pre_xr.velsurf_mag.sel(time=yrs[2]).plot(ax=axes[2,0], vmin=vmin, vmax=vmax,cmap=cmap)
y_true.sel(time=yrs[2]).plot(ax=axes[2,1], vmin=vmin, vmax=vmax,cmap=cmap)


for i, ax in enumerate(axes.flat):
    # left column: model prediction
    if i % 2 == 0:
        ax.set_title(f'tas model prediction (year = {yrs[i//2]})',fontweight='bold')
    # right column: truth tas from ssp245 simulations
    else:
        ax.set_title(f'tas truth (year = {yrs[i//2]})',fontweight='bold')
plt.tight_layout()